In [ ]:
#This create data size appropriate to run keras model. The result turns out to be 80% acuracy, which is not too bad!
#running the model through more data would improve accuracy, but will take a lot of time.
#code model run for 20 minutes through each itinery on a normal intel core 5 8 RAM latop. thousands of data would require a super computer to be fast.
#stronger processor is recommended.

import pickle
from scipy import sparse
from tensorflow import keras
from tensorflow.keras import layers, models
import numpy as np
import spektral
import os
import tensorflow as tf
import pandas
import matplotlib.pyplot as plt
import keras_tuner as kt

In [ ]:
#load all data made on organize_data_for_gnn into here
with open(r"C:\Users\buing\Documents\school research things\New research (machine learning)\wc_hydroxy-undecanethiol_k1.0.pickle", 'rb') as f:
    node = pickle.load(f)
    adj = pickle.load(f)
    y = pickle.load(f)

len(y)

In [3]:
from sklearn.model_selection import train_test_split
#make train/test list
n = np.arange(int(len(y) / 200))
train_ind, test_ind = train_test_split(n, random_state=0, test_size=0.2)
test_ind
#make 80% train size and 20% test size.

array([10,  1,  8, 18])

In [4]:
#create empty array of node/adj/energy value results.
node_train, adj_train, y_train = [], [], []
node_test, adj_test, y_test = [], [], []

In [5]:
#push interval of node/adj/hydrogen value with size 200 for testing batch.
for ind in train_ind:
    node_train.append(node[ind * 200 : (ind + 1) * 200])
    adj_train.append(adj[ind * 200 : (ind + 1) * 200])
    y_train.append(y[ind * 200 : (ind + 1) * 200])
    
for ind in test_ind:
    node_test.append(node[ind * 200 : (ind + 1) * 200])
    adj_test.append(adj[ind * 200 : (ind + 1) * 200])
    y_test.append(y[ind * 200 : (ind + 1) * 200])
    
import itertools
node_train = list(itertools.chain(*node_train))
adj_train = list(itertools.chain(*adj_train))
y_train = list(itertools.chain(*y_train))
node_test = list(itertools.chain(*node_test))
adj_test = list(itertools.chain(*adj_test))
y_test = list(itertools.chain(*y_test))

In [6]:
node_train = np.random.RandomState(0).permutation(node_train)
adj_train = np.random.RandomState(0).permutation(adj_train)
y_train = np.random.RandomState(0).permutation(y_train)


In [7]:
class DataGenerator(keras.utils.Sequence):
    # Generate data for keras
    def __init__(self, data, batch_size=15, shuffle=True, validation =True, test = False, demonstration = False ): #validation also = train
        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.demonstration = demonstration
        if demonstration:
            self.list_IDs_all = np.arange(100)
        else:
            self.list_IDs_all = np.arange(len(data[0]))
        self.validation = validation # = self.train
        self.test = test
        if test:
            self.list_IDs = self.list_IDs_all
        else:
            if self.validation:
                _, self.list_IDs = train_test_split(self.list_IDs_all, random_state=0, test_size=0.2)
            else:
                self.list_IDs, _ = train_test_split(self.list_IDs_all, random_state=0, test_size=0.2)
        self.indices = np.arange(len(self.list_IDs))
        
        #self.train = train
        #if self.train:
        #    self.list_IDs, _ = train_test_split(self.list_IDs_all, random_state=0, test_size=0.2)
        #else:
        #    _, self.list_IDs = train_test_split(self.list_IDs_all, random_state=0, test_size=0.2)
        #self.indices = np.arange(len(self.list_IDs))
    
    def __len__(self):
        # denote the number of batches per epoch
        return int(np.ceil(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        # generate one batch of data
        ind = self.indices[index * self.batch_size : (index + 1) * self.batch_size]
        list_IDs_temp = [self.list_IDs[i] for i in ind]
        
        # generate data
        x, y = self.__data_generation(list_IDs_temp)
        return x, y
    
    def on_epoch_end(self):
        # updates indices after each epoch
        if self.shuffle:
            np.random.shuffle(self.indices)
    
    def __data_generation(self, list_IDs_temp):
        # generate data containing batch_size samples
        node = []
        adj = []
        y = []
        
        for i in list_IDs_temp:
            node_temp = self.data[0][i].toarray() # convert sparse array back to dense array
            adj_temp = self.data[1][i].toarray()
            node.append(node_temp)
            adj.append(adj_temp)
            y.append(self.data[2][i])
        node = np.array(node)
        adj = np.array(adj)
        adj = spektral.utils.convolution.gcn_filter(adj)
        y = np.array(y)
        return [node, adj], y
        

In [24]:
batch_size = 15
train_generator = DataGenerator(data=[node_train, adj_train, y_train], 
                                    batch_size=batch_size, 
                                    shuffle=True, 
                                    validation=False, 
                                    test=False, 
                                    demonstration=False)
    
valid_generator = DataGenerator(data=[node_train, adj_train, y_train], 
                                    batch_size=batch_size, 
                                    shuffle=True, 
                                    validation=True, 
                                    test=False,
                                    demonstration=False)
    
test_generator = DataGenerator(data=[node_train, adj_train, y_train], 
                                   batch_size=batch_size, 
                                   shuffle=True, 
                                   validation=None, 
                                   test=True,
                                   demonstration=False)

In [9]:
test_generator[0][0][0].shape

(15, 2280, 7)

In [17]:
def model_builder(hp):
    x_node = layers.Input(shape=(2280, 7))
    x_adj = layers.Input(shape=(2280, 2280))
    #hyper-parameter 1
    # Tune the number of channels in the GCN layers
    # Choose an optimal value between 8-64
    hp_units_1 = hp.Int('channels', min_value=8, max_value=64, step=8)
    #change channels = value seen, since it will take forever.
    x = spektral.layers.GCNConv(channels=hp_units_1, activation='relu')([x_node, x_adj])
    x = spektral.layers.GCNConv(channels=hp_units_1, activation='relu')([x, x_adj])
    
    #hyper-parameter 2
    # Tune the number of channels in the Global Attention Pool
    # Choose an optimal value between 8-64
    hp_units_2 = hp.Int('channels', min_value=8, max_value=64, step=8)
    x = spektral.layers.GlobalAttentionPool(channels = hp_units_2)(x)
    x = layers.Flatten()(x)
    
    #hyper-parameter 3
    # Tune the number of units in the Dense layers
    # Choose an optimal value between 8-64
    hp_units_3 = hp.Int('units', min_value=8, max_value=64, step=8)
    
    x = layers.Dense(units=hp_units_3, activation='relu')(x)
    x = layers.Dense(units=hp_units_3, activation='relu')(x)
    x = layers.Dense(1, activation='linear')(x)
    
    model = models.Model(inputs=[x_node, x_adj], outputs=x)
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate= hp_learning_rate), 
                                                  loss='mse', metrics=['mae'])
    #model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    #filename = r"\GCN_checkpointrun.sav"
    #pickle.dump(model, open(filename, 'wb'))
    return model

In [22]:
#the model to run
x_node = layers.Input(shape=(2280, 7))
x_adj = layers.Input(shape=(2280, 2280))
    #hyper-parameter 1
    # Tune the number of channels in the GCN layers
    # Choose an optimal value between 8-64
    #change channels = value seen, since it will take forever.
x = spektral.layers.GCNConv(channels=8, activation='relu')([x_node, x_adj])
x = spektral.layers.GCNConv(channels=8, activation='relu')([x, x_adj])
    
    #hyper-parameter 2
    # Tune the number of channels in the Global Attention Pool
    # Choose an optimal value between 8-64
x = spektral.layers.GlobalAttentionPool(channels = 8)(x)
x = layers.Flatten()(x)
    
    #hyper-parameter 3
    # Tune the number of units in the Dense layers
    # Choose an optimal value between 8-64
x = layers.Dense(units=16, activation='relu')(x)
x = layers.Dense(units=16, activation='relu')(x)
x = layers.Dense(1, activation='linear')(x)
    
model = models.Model(inputs=[x_node, x_adj], outputs=x)
    
model.compile(optimizer=keras.optimizers.Adam(learning_rate= 1e-2), 
                                                loss='mse', metrics=['mae'])

In [11]:
# Model weights are saved at the end of every epoch, if it's the best seen
# so far.
#hist = model.fit(training_generator, validation_data=validation_generator, callbacks=[model_checkpoint_callback], epochs=1)
#filename = r"\GCN_checkpointrun.sav"
#pickle.dump(model, open(filename, 'wb'))
# The model weights (that are considered the best) are loaded into the model.
#model.load_weights(checkpoint_filepath)

TUNER

In [14]:
tuner = kt.Hyperband(model_builder,
                     objective='val_loss',
                     max_epochs=10,
                     factor=3,
                     directory='./',
                     project_name='gnn_hyperparameter')
#implement stop early
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_generator, epochs=50, validation_data=valid_generator, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 5 Complete [00h 06m 36s]
val_loss: 42196.41796875

Best val_loss So Far: 329.65924072265625
Total elapsed time: 00h 15m 16s

Search: Running Trial #6

Hyperparameter    |Value             |Best Value So Far 
channels          |32                |8                 
units             |16                |16                
learning_rate     |0.01              |0.01              
tuner/epochs      |10                |2                 
tuner/initial_e...|4                 |0                 
tuner/bracket     |2                 |2                 
tuner/round       |2                 |0                 
tuner/trial_id    |3d879bb76d04155...|None              

Epoch 5/10
6/6 [==============================] - 73s 13s/step - loss: 213397.1250 - mae: 358.2066 - val_loss: 39378.9648 - val_mae: 195.5854
Epoch 6/10
6/6 [==============================] - 64s 11s/step - loss: 20385.3867 - mae: 125.7165 - val_loss: 12489.6816 - val_mae: 109.3204
Epoch 7/10
6/6 [==============================

KeyboardInterrupt: 

In [16]:
#pickle.dump(hist.history, open(r"C:\Users\buing\Documents\school research things\New research (machine learning)\GCN_checkpointrun", 'wb'))
#loaded_model = pickle.load(open(r"C:\Users\buing\Documents\school research things\New research (machine learning)\GCN_checkpointrun", 'rb'))
#loaded_model = model.fit(training_generator, validation_data=validation_generator, callbacks=[model_checkpoint_callback], epochs=4)
#current_model = loaded_model


NameError: name 'best_hps' is not defined

set checkpoint

In [ ]:
checkpoint_filepath = os.path.join(r"C:\Users\buing\Documents\school research things\New research (machine learning)\GCN_checkpointrun.sav")
model_checkpoint = keras.callbacks.ModelCheckpoint(filepath= checkpoint_filepath, monitor='val_loss', mode='min',save_weights_only=False,save_best_only=False)
#model = tuner.hypermodel.build(best_hps)
history = model.fit(train_generator, epochs=50, validation_data=valid_generator, callbacks=[model_checkpoint])

#checkpoint_filepath = os.path.join(r"C:\Users\buing\Documents\school research things\New research (machine learning)\GCN_checkpointrun.sav")
#model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#    filepath= checkpoint_filepath,
 #   save_weights_only=True,
 #   monitor='val_accuracy',
  #  mode='max',
  #  save_best_only=False)